In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = './gdrive/My Drive/Colab Notebooks/bd'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


**Modelo: ResNet**

In [0]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, Dropout, Activation
from keras.layers import AveragePooling2D, Input, Flatten, MaxPooling2D,GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD
from keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.metrics import top_k_categorical_accuracy
import numpy as np
from scipy import ndimage
import h5py
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50 

import os
import glob

import keras
import h5py
import os
import numpy as np
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding


ext_img = '.jpg'
num_px = 75
num_py = 75
total_imgs = 25000     # total of images considering all classes
base_name = '/content/gdrive/My Drive/Colab Notebooks/scene_recognition/bd' # main folder
directory_results = '/content/gdrive/My Drive/Colab Notebooks/scene_recognition/Resultados/'
model_name= 'resnetCI'


num_classes = 10
batch_size = 128
epochs = 200

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Load data
hf = h5py.File(os.path.join(base_name, 'places%dpx_%dclass_2.h5' % (num_py, num_classes)), 'r')
x_train = np.array(hf.get('x_train'))
x_test = np.array(hf.get('x_test'))
y_train = np.array(hf.get('y_train'))
y_test = np.array(hf.get('y_test'))


labels_dic = {}
for l in hf.keys():
    if 'x_train' not in l and 'x_test' not in l and 'y_train' not in l and 'y_test' not in l:
        print(hf[l].value)
        print(l)
        labels_dic[hf[l].value] = l

hf.close()
del hf
print("Train shape" + str(x_train.shape))
print("Test shape" + str(x_test.shape))
print("Label test: " + str(labels_dic))


# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
#if subtract_pixel_mean:
#    x_train_mean = np.mean(x_train, axis=0)
#    x_train -= x_train_mean
#    x_test -= x_train_mean

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)


# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, 3) 
def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, 2) 

base_model = ResNet50(include_top=False, weights='imagenet',input_shape=input_shape,classes=num_classes)#weights=imagenet

model = base_model.output
model = GlobalAveragePooling2D()(model)
model = Dense(200, activation='relu')(model)
model = Dropout(0.5)(model)
predictions = Dense(num_classes, activation='softmax',kernel_initializer='he_normal')(model)
model = Model(inputs=base_model.input, outputs=predictions)



model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=lr_schedule(0), momentum=0.9),
              metrics=['accuracy',top_3_accuracy,top_2_accuracy])

print(model.summary())
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None
)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=None)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow(x_train, y_train,
        batch_size=batch_size)

# this is a similar generator, for validation data
validation_generator = test_datagen.flow(x_test, y_test,
        batch_size=batch_size)
lr_scheduler = LearningRateScheduler(lr_schedule)
callbacks =  [lr_schedule]
h = model.fit_generator(
        train_generator,
        steps_per_epoch=x_train.shape[0] // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=x_test.shape[0] // batch_size,
        callbacks=[LearningRateScheduler(lr_schedule)]
)



##Cria uma nova pasta para a respectiva execução

if os.path.exists(directory_results+model_name) == False:
  os.mkdir(directory_results+model_name)
     
directory_results = directory_results+model_name
tamanho_directory = len(directory_results)+1

lista = [os.path.join(directory_results, o) for o in os.listdir(directory_results) 
                    if os.path.isdir(os.path.join(directory_results,o))]
count = 0
for pasta in lista:
  count=int(pasta[tamanho_directory:])

count+=1
os.mkdir(directory_results+'/'+str(count))
directory_results = directory_results+'/'+str(count)+'/'






model.save_weights(os.path.join(directory_results,model_name+'.h5'))#save weights

print(h)
print(h.history)
# History of acc(train) and val_acc(validation)
hist_val_top_3_accuracy = h.history['val_top_3_accuracy']
hist_val_top_2_accuracy = h.history['val_top_2_accuracy']
hist_top_3_accuracy = h.history['top_3_accuracy']
hist_top_2_accuracy = h.history['top_2_accuracy']
hist_lr = h.history['lr']
hist_acc_train = h.history['acc']
hist_loss_train = h.history['loss']
hist_acc_test = h.history['val_acc']
hist_loss_test = h.history['val_loss']

print('History of train\n', hist_acc_train)
print('History of test\n', hist_acc_test)

np.save(directory_results+model_name+'_val_top_3_accuracy.npy',hist_val_top_3_accuracy)
np.save(directory_results+model_name+'_val_top_2_accuracy.npy',hist_val_top_2_accuracy)
np.save(directory_results+model_name+'_top_3_accuracy.npy',hist_top_3_accuracy)
np.save(directory_results+model_name+'_top_2_accuracy.npy',hist_top_2_accuracy)
np.save(directory_results+model_name+'_lr.npy',hist_lr)
np.save(directory_results+model_name+'_acc_train.npy',hist_acc_train)
np.save(directory_results+model_name+'_loss_train.npy',hist_loss_train)
np.save(directory_results+model_name+'_acc_test.npy',hist_acc_test)
np.save(directory_results+model_name+'_loss_test.npy',hist_loss_test)



# plot history
plt.figure(1)
plt.plot(hist_loss_train)
plt.plot(hist_loss_test)
plt.legend(['Train loss','Test loss'], loc='upper right')
plt.title('Learning curves')

plt.figure(2)
plt.plot(hist_acc_train)
plt.plot(hist_acc_test)
plt.legend(['Train acc','Test acc'], loc='upper right')
plt.title('Accuracy')

# Report
pred_test = np.argmax(model.predict(x_test, batch_size=1, verbose=0), axis=1)
true_test = np.argmax(y_test, axis=1)
cf = confusion_matrix(true_test, pred_test)
print(cf)

np.save(directory_results+model_name+'_pred_test.npy',pred_test)
np.save(directory_results+model_name+'_true_test.npy',true_test)
np.save(directory_results+model_name+'_cf.npy',cf)


if(not os.path.exists(directory_results+model_name+'.txt')):
  file = open(directory_results+model_name+'.txt', 'w')
  file.close()

file = open(directory_results+model_name+'.txt', 'r')

lines = file.readlines()

results_class = []
for cl in range(num_classes):
  print("\n%s: %.2f%%" % (labels_dic[cl], cf[cl,cl]/sum(true_test==cl)*100))
  lines.append("\n%s: %.2f%%" % (labels_dic[cl], cf[cl,cl]/sum(true_test==cl)*100))
  results_class.append(cf[cl,cl]/sum(true_test==cl)*100)

np.save(directory_results+model_name+'_results_class.npy',results_class)
labels = []
for i in labels_dic:
  labels.append(labels_dic[i])
np.save(directory_results+model_name+'_labels.npy',labels)

print(labels_dic.values())
print(classification_report(true_test, pred_test, target_names=labels_dic.values()))
lines.append("\n"+str(classification_report(true_test, pred_test, target_names=labels_dic.values())))

# save file execution
file = open(directory_results+model_name+'.txt', 'w')
file.writelines(lines)
file.close()


Using TensorFlow backend.
